In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import cm
import torchvision
from torch.utils import data
from torchvision import transforms

In [17]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [18]:
class BasicBlock(nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out



class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def project1_model():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# def test():
#     net = ResNet18()
#     y = net(torch.randn(1, 3, 32, 32))
#     print(y.size())


In [19]:
transform_train = torchvision.transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified


In [20]:
torch.cuda.is_available()

True

In [21]:
net = project1_model()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

train_loss_list = []
train_acc_list = []
test_loss_list = []
test_acc_list = []

In [22]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    batch_index = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        # loss = nll(outputs, targets)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        batch_index = batch_idx
        # train_loss += loss.item()
        train_loss = loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        print("\n")
    train_loss_list.append(train_loss/(batch_index+1))
    train_acc_list.append(100.*correct/total)

def test(epoch):
#     global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    batch_index = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            # loss = nll(outputs, targets)
            batch_index = batch_idx
            # test_loss += loss.item()
            test_loss = loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            print("\n")
    test_loss_list.append(test_loss/(batch_index+1))
    test_acc_list.append(100.*correct/total)

    # Save checkpoint.
#     acc = 100.*correct/total
#     if acc > best_acc:
#         print('Saving..')
#         state = {
#             'net': net.state_dict(),
#             'acc': acc,
#             'epoch': epoch,
#             'train_history': (train_loss_list, train_acc_list, test_loss_list, test_acc_list)
#         }
#         if not os.path.isdir('checkpoint'):
#             os.mkdir('checkpoint')
#         torch.save(state, './checkpoint/ckpt_rmsprop.pth')
#         best_acc = acc

In [25]:
for epoch in range(20):
    train(epoch)
    test(epoch)


Epoch: 0
0 391 Loss: 0.948 | Acc: 66.406% (85/128)


1 391 Loss: 0.528 | Acc: 64.453% (165/256)


2 391 Loss: 0.318 | Acc: 64.583% (248/384)


3 391 Loss: 0.249 | Acc: 64.453% (330/512)


4 391 Loss: 0.170 | Acc: 65.625% (420/640)


5 391 Loss: 0.149 | Acc: 66.797% (513/768)


6 391 Loss: 0.119 | Acc: 67.857% (608/896)


7 391 Loss: 0.114 | Acc: 67.773% (694/1024)


8 391 Loss: 0.096 | Acc: 67.795% (781/1152)


9 391 Loss: 0.094 | Acc: 67.578% (865/1280)


10 391 Loss: 0.068 | Acc: 68.253% (961/1408)


11 391 Loss: 0.071 | Acc: 68.164% (1047/1536)


12 391 Loss: 0.075 | Acc: 68.209% (1135/1664)


13 391 Loss: 0.062 | Acc: 68.192% (1222/1792)


14 391 Loss: 0.058 | Acc: 68.333% (1312/1920)


15 391 Loss: 0.053 | Acc: 68.262% (1398/2048)


16 391 Loss: 0.056 | Acc: 68.336% (1487/2176)


17 391 Loss: 0.050 | Acc: 68.186% (1571/2304)


18 391 Loss: 0.048 | Acc: 68.174% (1658/2432)


19 391 Loss: 0.043 | Acc: 68.047% (1742/2560)


20 391 Loss: 0.034 | Acc: 68.452% (1840/2688)


21 391 Loss

167 391 Loss: 0.005 | Acc: 68.118% (14648/21504)


168 391 Loss: 0.007 | Acc: 68.089% (14729/21632)


169 391 Loss: 0.005 | Acc: 68.130% (14825/21760)


170 391 Loss: 0.005 | Acc: 68.161% (14919/21888)


171 391 Loss: 0.005 | Acc: 68.187% (15012/22016)


172 391 Loss: 0.005 | Acc: 68.172% (15096/22144)


173 391 Loss: 0.004 | Acc: 68.189% (15187/22272)


174 391 Loss: 0.004 | Acc: 68.232% (15284/22400)


175 391 Loss: 0.005 | Acc: 68.226% (15370/22528)


176 391 Loss: 0.004 | Acc: 68.242% (15461/22656)


177 391 Loss: 0.006 | Acc: 68.236% (15547/22784)


178 391 Loss: 0.005 | Acc: 68.274% (15643/22912)


179 391 Loss: 0.004 | Acc: 68.307% (15738/23040)


180 391 Loss: 0.005 | Acc: 68.305% (15825/23168)


181 391 Loss: 0.005 | Acc: 68.291% (15909/23296)


182 391 Loss: 0.005 | Acc: 68.289% (15996/23424)


183 391 Loss: 0.005 | Acc: 68.266% (16078/23552)


184 391 Loss: 0.004 | Acc: 68.281% (16169/23680)


185 391 Loss: 0.005 | Acc: 68.317% (16265/23808)


186 391 Loss: 0.004 | Acc: 68.3

330 391 Loss: 0.003 | Acc: 69.319% (29369/42368)


331 391 Loss: 0.003 | Acc: 69.305% (29452/42496)


332 391 Loss: 0.002 | Acc: 69.315% (29545/42624)


333 391 Loss: 0.003 | Acc: 69.309% (29631/42752)


334 391 Loss: 0.003 | Acc: 69.300% (29716/42880)


335 391 Loss: 0.003 | Acc: 69.306% (29807/43008)


336 391 Loss: 0.002 | Acc: 69.306% (29896/43136)


337 391 Loss: 0.003 | Acc: 69.307% (29985/43264)


338 391 Loss: 0.002 | Acc: 69.317% (30078/43392)


339 391 Loss: 0.003 | Acc: 69.304% (30161/43520)


340 391 Loss: 0.002 | Acc: 69.314% (30254/43648)


341 391 Loss: 0.002 | Acc: 69.312% (30342/43776)


342 391 Loss: 0.002 | Acc: 69.315% (30432/43904)


343 391 Loss: 0.002 | Acc: 69.343% (30533/44032)


344 391 Loss: 0.002 | Acc: 69.332% (30617/44160)


345 391 Loss: 0.003 | Acc: 69.328% (30704/44288)


346 391 Loss: 0.002 | Acc: 69.326% (30792/44416)


347 391 Loss: 0.003 | Acc: 69.316% (30876/44544)


348 391 Loss: 0.002 | Acc: 69.319% (30966/44672)


349 391 Loss: 0.002 | Acc: 69.3

109 391 Loss: 0.008 | Acc: 71.996% (10137/14080)


110 391 Loss: 0.007 | Acc: 71.995% (10229/14208)


111 391 Loss: 0.006 | Acc: 72.084% (10334/14336)


112 391 Loss: 0.008 | Acc: 72.041% (10420/14464)


113 391 Loss: 0.007 | Acc: 72.039% (10512/14592)


114 391 Loss: 0.006 | Acc: 72.058% (10607/14720)


115 391 Loss: 0.007 | Acc: 72.037% (10696/14848)


116 391 Loss: 0.007 | Acc: 72.049% (10790/14976)


117 391 Loss: 0.006 | Acc: 72.127% (10894/15104)


118 391 Loss: 0.008 | Acc: 72.105% (10983/15232)


119 391 Loss: 0.006 | Acc: 72.135% (11080/15360)


120 391 Loss: 0.005 | Acc: 72.198% (11182/15488)


121 391 Loss: 0.005 | Acc: 72.259% (11284/15616)


122 391 Loss: 0.006 | Acc: 72.269% (11378/15744)


123 391 Loss: 0.007 | Acc: 72.259% (11469/15872)


124 391 Loss: 0.006 | Acc: 72.269% (11563/16000)


125 391 Loss: 0.005 | Acc: 72.334% (11666/16128)


126 391 Loss: 0.006 | Acc: 72.281% (11750/16256)


127 391 Loss: 0.005 | Acc: 72.333% (11851/16384)


128 391 Loss: 0.006 | Acc: 72.3

271 391 Loss: 0.002 | Acc: 72.969% (25405/34816)


272 391 Loss: 0.002 | Acc: 72.985% (25504/34944)


273 391 Loss: 0.003 | Acc: 72.981% (25596/35072)


274 391 Loss: 0.003 | Acc: 72.969% (25685/35200)


275 391 Loss: 0.003 | Acc: 72.973% (25780/35328)


276 391 Loss: 0.003 | Acc: 72.975% (25874/35456)


277 391 Loss: 0.003 | Acc: 72.971% (25966/35584)


278 391 Loss: 0.002 | Acc: 73.009% (26073/35712)


279 391 Loss: 0.003 | Acc: 73.016% (26169/35840)


280 391 Loss: 0.002 | Acc: 73.034% (26269/35968)


281 391 Loss: 0.003 | Acc: 73.030% (26361/36096)


282 391 Loss: 0.002 | Acc: 73.065% (26467/36224)


283 391 Loss: 0.003 | Acc: 73.052% (26556/36352)


284 391 Loss: 0.003 | Acc: 73.046% (26647/36480)


285 391 Loss: 0.003 | Acc: 73.017% (26730/36608)


286 391 Loss: 0.002 | Acc: 73.021% (26825/36736)


287 391 Loss: 0.003 | Acc: 73.031% (26922/36864)


288 391 Loss: 0.003 | Acc: 73.024% (27013/36992)


289 391 Loss: 0.002 | Acc: 73.039% (27112/37120)


290 391 Loss: 0.003 | Acc: 73.0

...

57 391 Loss: 0.004 | Acc: 91.150% (6767/7424)


58 391 Loss: 0.004 | Acc: 91.155% (6884/7552)


59 391 Loss: 0.002 | Acc: 91.237% (7007/7680)


60 391 Loss: 0.006 | Acc: 91.163% (7118/7808)


61 391 Loss: 0.003 | Acc: 91.217% (7239/7936)


62 391 Loss: 0.004 | Acc: 91.220% (7356/8064)


63 391 Loss: 0.005 | Acc: 91.125% (7465/8192)


64 391 Loss: 0.004 | Acc: 91.130% (7582/8320)


65 391 Loss: 0.002 | Acc: 91.205% (7705/8448)


66 391 Loss: 0.004 | Acc: 91.185% (7820/8576)


67 391 Loss: 0.004 | Acc: 91.165% (7935/8704)


68 391 Loss: 0.003 | Acc: 91.191% (8054/8832)


69 391 Loss: 0.002 | Acc: 91.250% (8176/8960)


70 391 Loss: 0.002 | Acc: 91.307% (8298/9088)


71 391 Loss: 0.004 | Acc: 91.298% (8414/9216)


72 391 Loss: 0.004 | Acc: 91.289% (8530/9344)


73 391 Loss: 0.003 | Acc: 91.248% (8643/9472)


74 391 Loss: 0.002 | Acc: 91.302% (8765/9600)


75 391 Loss: 0.003 | Acc: 91.334% (8885/9728)


76 391 Loss: 0.002 | Acc: 91.356% (9004/9856)


77 391 Loss: 0.003 | Acc: 91.346% (9120/

222 391 Loss: 0.001 | Acc: 91.564% (26136/28544)


223 391 Loss: 0.001 | Acc: 91.567% (26254/28672)


224 391 Loss: 0.001 | Acc: 91.573% (26373/28800)


225 391 Loss: 0.001 | Acc: 91.593% (26496/28928)


226 391 Loss: 0.001 | Acc: 91.599% (26615/29056)


227 391 Loss: 0.001 | Acc: 91.571% (26724/29184)


228 391 Loss: 0.001 | Acc: 91.567% (26840/29312)


229 391 Loss: 0.001 | Acc: 91.579% (26961/29440)


230 391 Loss: 0.001 | Acc: 91.572% (27076/29568)


231 391 Loss: 0.001 | Acc: 91.578% (27195/29696)


232 391 Loss: 0.001 | Acc: 91.581% (27313/29824)


233 391 Loss: 0.001 | Acc: 91.580% (27430/29952)


234 391 Loss: 0.001 | Acc: 91.589% (27550/30080)


235 391 Loss: 0.001 | Acc: 91.585% (27666/30208)


236 391 Loss: 0.001 | Acc: 91.584% (27783/30336)


237 391 Loss: 0.001 | Acc: 91.584% (27900/30464)


238 391 Loss: 0.001 | Acc: 91.579% (28016/30592)


239 391 Loss: 0.001 | Acc: 91.579% (28133/30720)


240 391 Loss: 0.001 | Acc: 91.575% (28249/30848)


241 391 Loss: 0.001 | Acc: 91.5

386 391 Loss: 0.001 | Acc: 91.263% (45208/49536)


387 391 Loss: 0.001 | Acc: 91.269% (45328/49664)


388 391 Loss: 0.001 | Acc: 91.268% (45444/49792)


389 391 Loss: 0.001 | Acc: 91.270% (45562/49920)


390 391 Loss: 0.001 | Acc: 91.270% (45635/50000)



Epoch: 19
0 391 Loss: 0.274 | Acc: 90.625% (116/128)


1 391 Loss: 0.111 | Acc: 91.016% (233/256)


2 391 Loss: 0.086 | Acc: 90.885% (349/384)


3 391 Loss: 0.056 | Acc: 91.016% (466/512)


4 391 Loss: 0.056 | Acc: 90.938% (582/640)


5 391 Loss: 0.038 | Acc: 91.276% (701/768)


6 391 Loss: 0.049 | Acc: 90.737% (813/896)


7 391 Loss: 0.023 | Acc: 91.113% (933/1024)


8 391 Loss: 0.035 | Acc: 90.712% (1045/1152)


9 391 Loss: 0.020 | Acc: 90.703% (1161/1280)


10 391 Loss: 0.024 | Acc: 90.625% (1276/1408)


11 391 Loss: 0.020 | Acc: 90.625% (1392/1536)


12 391 Loss: 0.021 | Acc: 90.685% (1509/1664)


13 391 Loss: 0.018 | Acc: 90.792% (1627/1792)


14 391 Loss: 0.015 | Acc: 90.885% (1745/1920)


15 391 Loss: 0.010 | Acc: 91.113% (1866

163 391 Loss: 0.002 | Acc: 91.525% (19213/20992)


164 391 Loss: 0.001 | Acc: 91.539% (19333/21120)


165 391 Loss: 0.002 | Acc: 91.529% (19448/21248)


166 391 Loss: 0.002 | Acc: 91.542% (19568/21376)


167 391 Loss: 0.002 | Acc: 91.518% (19680/21504)


168 391 Loss: 0.001 | Acc: 91.517% (19797/21632)


169 391 Loss: 0.001 | Acc: 91.535% (19918/21760)


170 391 Loss: 0.002 | Acc: 91.534% (20035/21888)


171 391 Loss: 0.001 | Acc: 91.547% (20155/22016)


172 391 Loss: 0.002 | Acc: 91.524% (20267/22144)


173 391 Loss: 0.002 | Acc: 91.514% (20382/22272)


174 391 Loss: 0.002 | Acc: 91.482% (20492/22400)


175 391 Loss: 0.001 | Acc: 91.499% (20613/22528)


176 391 Loss: 0.001 | Acc: 91.508% (20732/22656)


177 391 Loss: 0.002 | Acc: 91.490% (20845/22784)


178 391 Loss: 0.001 | Acc: 91.485% (20961/22912)


179 391 Loss: 0.002 | Acc: 91.480% (21077/23040)


180 391 Loss: 0.001 | Acc: 91.501% (21199/23168)


181 391 Loss: 0.001 | Acc: 91.514% (21319/23296)


182 391 Loss: 0.001 | Acc: 91.5

327 391 Loss: 0.001 | Acc: 91.578% (38448/41984)


328 391 Loss: 0.001 | Acc: 91.577% (38565/42112)


329 391 Loss: 0.001 | Acc: 91.577% (38682/42240)


330 391 Loss: 0.001 | Acc: 91.583% (38802/42368)


331 391 Loss: 0.001 | Acc: 91.583% (38919/42496)


332 391 Loss: 0.001 | Acc: 91.585% (39037/42624)


333 391 Loss: 0.001 | Acc: 91.589% (39156/42752)


334 391 Loss: 0.000 | Acc: 91.595% (39276/42880)


335 391 Loss: 0.001 | Acc: 91.609% (39399/43008)


336 391 Loss: 0.001 | Acc: 91.613% (39518/43136)


337 391 Loss: 0.001 | Acc: 91.605% (39632/43264)


338 391 Loss: 0.000 | Acc: 91.611% (39752/43392)


339 391 Loss: 0.001 | Acc: 91.602% (39865/43520)


340 391 Loss: 0.001 | Acc: 91.606% (39984/43648)


341 391 Loss: 0.000 | Acc: 91.614% (40105/43776)


342 391 Loss: 0.001 | Acc: 91.614% (40222/43904)


343 391 Loss: 0.000 | Acc: 91.629% (40346/44032)


344 391 Loss: 0.001 | Acc: 91.626% (40462/44160)


345 391 Loss: 0.001 | Acc: 91.628% (40580/44288)


346 391 Loss: 0.001 | Acc: 91.6

In [26]:
print(train_acc_list)

[40.164, 56.136, 64.12, 69.608, 73.522, 76.562, 78.644, 80.588, 82.004, 83.098, 84.022, 85.254, 86.328, 86.912, 87.634, 88.234, 88.614, 89.394, 89.846, 90.472, 90.774, 91.27, 91.56]
